In [1]:
from datetime import datetime
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from chinese_calendar import is_workday, is_holiday
#读取picke文件
import pickle

def stationID(x):
    new_dict = {v : k for k, v in station_dict.items()}
    return new_dict[x]
station_dict = {
    0 : '湘湖',
    1 : '滨康路',
    2 : '西兴',
    3 : '滨和路',
    4 : '江陵路',
    5 : '近江',
    6 : '婺江路',
    7 : '城站',
    8 : '定安路',
    9 : '龙翔桥',
    10 : '凤起路',
    11 : '武林广场',
    12 : '西湖文化广场',
    13 : '打铁关',
    14 : '闸弄口',
    15 : '火车东站',
    16 : '彭埠',
    17 : '七堡',
    18 : '九和路',
    19 : '九堡',
    20 : '客运中心',
    21 : '下沙西',
    22 : '金沙湖',
    23 : '高沙路',
    24 : '文泽路',
    25 : '文海南路',
    26 : '云水',
    27 : '下沙江滨',
    28 : '乔司南',
    29 : '乔司',
    30 : '翁梅',
    31 : '余杭高铁站',
    32 : '南苑',
    33 : '临平',
    34 : '朝阳',
    35 : '曹家桥',
    36 : '潘水',
    37 : '人民路',
    38 : '杭发厂',
    39 : '人民广场',
    40 : '建设一路',
    41 : '建设三路',
    42 : '振宁路',
    43 : '飞虹路',
    44 : '盈丰路',
    45 : '钱江世纪城',
    46 : '钱江路',
    47 : '庆春广场',
    48 : '庆菱路',
    49 : '建国北路',
    50 : '中河北路',
    51 : '凤起路',
    52 : '武林门',
    53 : '沈塘桥',
    54 : '下宁桥',
    55 : '学院路',
    56 : '古翠路',
    57 : '丰潭路',
    58 : '文新',
    59 : '三坝',
    60 : '虾龙圩',
    61 : '三墩',
    62 : '墩祥街',
    63 : '金家渡',
    64 : '白洋',
    65 : '杜甫村',
    66 : '良渚',
    67 : '浦沿',
    68 : '杨家墩',
    69 : '中医药大学',
    70 : '联庄',
    71 : '水澄桥',
    72 : '复兴路',
    73 : '南星桥',
    74 : '甬江路',
    75 : '城星路',
    76 : '市民中心',
    77 : '江锦路',
    78 : '景芳',
    79 : '新塘',
    80 : '新风'
}

with open('../odflow_1s.pk', 'rb') as file:   #用with的优点是可以不用写关闭文件操作
    odflow_1s = pickle.load(file)

In [29]:
odflow_1s['2019-01-01 07:17:00'].keys()

dict_keys(['白洋', '南苑', '客运中心', '文泽路', '盈丰路', '火车东站', '杜甫村', '乔司', '联庄', '新风', '湘湖', '滨和路', '定安路', '振宁路', '九和路', '沈塘桥', '飞虹路', '金家渡', '下沙江滨', '临平', '武林广场', '云水', '杨家墩', '朝阳', '钱江路', '三墩', '虾龙圩', '浦沿', '闸弄口', '水澄桥', '西兴', '武林门', '城站', '良渚', '丰潭路', '滨康路', '南星桥', '文新', '龙翔桥', '彭埠', '城星路', '西湖文化广场', '金沙湖', '复兴路', '古翠路', '七堡', '江陵路', '建设一路', '曹家桥', '中医药大学', '打铁关', '市民中心', '学院路', '凤起路', '余杭高铁站', '高沙路', '墩祥街', '乔司南', '建设三路', '近江', '三坝', '新塘', '人民广场', '文海南路', '钱江世纪城', '中河北路', '九堡', '建国北路', '翁梅', '江锦路', '婺江路', '景芳', '潘水', '庆菱路', '甬江路', '人民路', '杭发厂', '下沙西'])

In [ ]:
freq=300
freq_str=str(freq)+'s'
freq_str
start = '2019-01-01 06:00:00'
end = '2019-01-25 23:59:00'
pdates= pd.date_range(start=start, end=end,freq=freq_str)
pdates

In [23]:
def delta_adj_generate(data_range):
    num_sensors = 81
    od_adj = np.zeros((num_sensors, num_sensors), dtype=np.float32)
    od_adj[:] = 0
    for i in range(data_range.shape[0]):
        time_list=data_range[i]
        if str(time_list) in list(odflow_1s.keys()):
            for j in odflow_1s[str(time_list)].keys():
                startid = stationID(j)
                for k in odflow_1s[str(time_list)][j]:
                    endid = stationID(k)
                    value = odflow_1s[str(time_list)][j][k]
                    od_adj[startid,endid]+=value
    return od_adj

In [ ]:
def generate_odadj(freq):
    freq_str=str(freq)+'s'
    starttime = datetime.now() 
    start = '2019-01-01 06:00:00'
    end = '2019-01-25 23:59:00'
    pdates= pd.date_range(start=start, end=end,freq=freq_str)
    
    adj = {}
    for data_start in pdates:
        time1 = datetime.now() 
        data_freq = '1s'
        data_end = data_start + pd.Timedelta(seconds=freq-1)
        data_range = pd.date_range(start=data_start, end=data_end,freq=data_freq)
        od_adj = delta_adj_generate(data_range)
        adj[data_start]=od_adj
        time2 = datetime.now()
        deltatime = (time2 - time1).seconds
        print('one pdates time uses: ', deltatime, ' seconds')
    endtime = datetime.now()
    alltime = (endtime - starttime).seconds
    print('alltime: ',alltime)
    hour = int(alltime/3600)
    minute = int((alltime-hour*3600)/60)
    second = alltime-hour*3600-minute*60
    print( "time used: ", hour, ' hours ',minute,' minutes', second,' second') 
    return adj

In [25]:
data_start= pdates[0]
data_freq = '1s'
freq=300
data_end = data_start + pd.Timedelta(seconds=freq-1)
# data_end = pdates[1] - pd.Timedelta(seconds=1)
data_range = pd.date_range(start=data_start, end=data_end,freq=data_freq)

num_sensors = 81
od_adj2 = np.zeros((num_sensors, num_sensors), dtype=np.float32)
od_adj2[:] = 0
for i in range(data_range.shape[0]):
    time_list=data_range[i]
    if str(time_list) in list(odflow_1s.keys()):
        for j in odflow_1s[str(time_list)].keys():
            startid = stationID(j)
            for k in odflow_1s[str(time_list)][j]:
                endid = stationID(k)
                value = odflow_1s[str(time_list)][j][k]
                od_adj2[startid,endid]+=value
od_adj2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

DatetimeIndex(['2019-01-01 06:00:00', '2019-01-01 06:05:00',
               '2019-01-01 06:10:00', '2019-01-01 06:15:00',
               '2019-01-01 06:20:00', '2019-01-01 06:25:00',
               '2019-01-01 06:30:00', '2019-01-01 06:35:00',
               '2019-01-01 06:40:00', '2019-01-01 06:45:00',
               ...
               '2019-01-25 23:10:00', '2019-01-25 23:15:00',
               '2019-01-25 23:20:00', '2019-01-25 23:25:00',
               '2019-01-25 23:30:00', '2019-01-25 23:35:00',
               '2019-01-25 23:40:00', '2019-01-25 23:45:00',
               '2019-01-25 23:50:00', '2019-01-25 23:55:00'],
              dtype='datetime64[ns]', length=7128, freq='300S')

In [17]:
def generate_odadj(freq):
    freq_str=str(freq)+'s'
    starttime = datetime.now() 
    start = '2019-01-01 06:00:00'
    end = '2019-01-25 23:59:00'
    pdates= pd.date_range(start=start, end=end,freq=freq_str)
    
    adj = {}
    for data_start in pdates:
        time1 = datetime.now() 
        data_freq = '1s'
        data_end = data_start + pd.Timedelta(seconds=freq-1)
        data_range = pd.date_range(start=data_start, end=data_end,freq=data_freq)
        od_adj = delta_adj_generate(data_range)
        adj[data_start]=od_adj
        time2 = datetime.now()
        deltatime = (time2 - time1).seconds
        print('one pdates time uses: ', deltatime, ' seconds')
    endtime = datetime.now()
    alltime = (endtime - starttime).seconds
    print('alltime: ',alltime)
    hour = int(alltime/3600)
    minute = alltime%3600
    print( "time used: ", hour, ' hours ',minute,' minutes') 
    return adj

In [18]:
od_adj_5m = generate_odadj('300s')

one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates tim

one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates tim

one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates tim

one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates tim

one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates tim

...

one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates tim

one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates tim

one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates tim

one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates tim

one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates time uses:  0  seconds
one pdates tim

In [19]:
len(list(od_adj_5m.keys()))

7128

In [25]:
od_adj_5m['2019-01-01 12:45:00']

KeyError: '2019-01-01 12:45:00'

In [16]:
adj

NameError: name 'adj' is not defined

In [22]:
def matrix_add(matrix1,matrix2):
    total_element = [matrix1[i][j] + matrix2[i][j] for i in range(len(matrix1)) for j in range(len(matrix1))]
    new_matrix = [total_element[x:x+len(matrix1)] for x in range(0,len(total_element),len(matrix1))]
    return new_matrix

matrix1 = [[1,2,3,1], [4,5,6,1], [7,8,9,1], [7,8,9,1]]
matrix2 = [[2,2,2,1], [3,3,3,1], [4,4,4,1], [7,8,9,1]]
new_matrix = matrix_add(matrix1, matrix2)
new_matrix

[[3, 4, 5, 2], [7, 8, 9, 2], [11, 12, 13, 2], [14, 16, 18, 2]]

In [26]:
def matrix_add(matrix1,matrix2):
    total_element = [matrix1[i][j] + matrix2[i][j] for i in range(len(matrix1)) for j in range(len(matrix1))]
    new_matrix = [total_element[x:x+len(matrix1)] for x in range(0,len(total_element),len(matrix1))]
    return new_matrix

num_sensors = 81
adj1 = np.zeros((num_sensors, num_sensors), dtype=np.float32)
adj1[:] = 0
start = '2019-01-01 12:00:00'
end = '2019-01-01 12:59:00'
onehour = pd.date_range(start=start, end=end,freq='300s')
for onestep in onehour:
    temp_adj=od_adj_5m[onestep]
    adj1 = matrix_add(adj1, temp_adj)
adj1

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0

In [33]:
pd.DataFrame(adj[pdates[0]])

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
